In [ ]:
################################################################################################
# <문장 임베딩 예시>
# sentence-bert와 Faiss 라이브러리를 이용하여 검색 MRR(Mean Reciprocal Rank)측정 예시 6
# - 문서들을 슬라이딩 방식으로 청크 -> 청크에서 문장들 분리-> 문장들 클러스터링 후 성능 비교
# - 쿼리문장 임베딩벡터를 구해서 각 contexts(문서) Faiss 인덱스에서 sub 문장 벡터와 비교하여 유사도 스코어 계산.
# - 말뭉치는  KorQuAD_v1.0 파일과 aihub(aihub.or.kr) 뉴스 기사 기계독해 데이터 QuA 파일 3종류 이용
#------------------------------------------------------------------------------
# 1. 검색모델 로딩
# 2. JSON 파일 로딩 후 df 만듬
# 3. 문서->청크 만들고->문장 만듬.
# 4. 분리된 문장들에 대해 클러스터링 수행후, 평귵 임베딩 구하고, Faiss에 인덱싱 함.
# 5. 쿼리 후 가장 유사한 문장(문서)의 sub 문장 평균 구함
# 6. MRR 계산

#-------------------------------------------------------------------------------
# sklenarn 으로 cosine 확인 예제
# from sklearn.metrics.pairwise import cosine_similarity
# cosine = cosine_similarity([embed_querys[0]], [embed_querys_1[0]]) # (1,768) 식에 2차원 배열입력되어야 함.
#-------------------------------------------------------------------------------
################################################################################################

import faiss
import numpy as np
import pandas as pd
import time

from os import sys
sys.path.append('../../')
from myutils import GPU_info, seed_everything, mlogging, bi_encoder

logger = mlogging(loggername="MRR-h", logfilename="../../../log/MRR-h")
device = GPU_info()
#device = 'cpu'  # cpu 테스트 할때

#------------------------------------------------------------------------------------
# 0. param 설정
#------------------------------------------------------------------------------------
seed = 111
query_num = 500             # 쿼리 최대 갯수: KorQuAD_v1.0_dev.json 최대값은 5533개임, 0이면 모든 5533개 쿼리함.
search_k = 5                # FAISS 검색시, 검색 계수(5=쿼리와 가장 근접한 5개 결과값을 반환함)
avg_num = 1                 # 쿼리에 대해 sub 문장들중 최대 scorce를 갖는 문장을 몇개 찾고 평균낼지.(3=쿼리에 가장 유사한 sub문장 3개를 찾고 평균을 냄)
faiss_index_method = 0      # 0= Cosine Similarity 적용(IndexFlatIP 사용), 1= Euclidean Distance 적용(IndexFlatL2 사용)

# 슬라이딩 윈도우 param
IS_SLIDING_WINDOW = False
WINDOW_SIZE=256             # 문단을 몇 token으로 나눌지          (128,0)=>78.40%, (256, 64)=>75.20%
SLIDING_SIZE=64             # 중첩되는 token 

# 클러스트링 param
IS_CLUSTERING = True
num_clusters = 5           # 클러스터링 계수 

IS_ONNX_MODEL = False
#------------------------------------------------------------------------------------

seed_everything(seed)

In [ ]:
#-------------------------------------------------------------------------------------
# 1. 검색모델 로딩
# => bi_encoder 모델 로딩, polling_mode 설정
# => bi_encoder1 = SentenceTransformer(bi_encoder_path) # 오히려 성능 떨어짐. 이유는 do_lower_case나, max_seq_len등 세부 설정이 안되므로.
#-------------------------------------------------------------------------------------
import torch
from myutils import bi_encoder, dense_model, onnx_model, onnx_embed_text
from sentence_transformers import SentenceTransformer

#bi_encoder_path = "../../../data11/model/moco/albert-small-distil/sbert-albert-small-sts-distil-32-last-nli-128d-sts" #"bongsoo/kpf-sbert-v1.1" # kpf-sbert-v1.1 # klue-sbert-v1 # albert-small-kor-sbert-v1.1
bi_encoder_path = "bongsoo/kpf-sbert-128d-v1"
pooling_mode = 'mean' # bert면=mean, albert면 = cls
out_dimension = 128  # 0이면 768임.

word_embedding_model1, bi_encoder1 = bi_encoder(model_path=bi_encoder_path, max_seq_len=512, do_lower_case=True, 
                                              pooling_mode=pooling_mode, out_dimension=out_dimension, device=device)
  
print(f'\n---bi_encoder---------------------------')
print(bi_encoder1)
print(word_embedding_model1)
#------------------------------------------------------------------------------------------------------------------------

In [ ]:
#-------------------------------------------------------------------------------------------------------
# 2. JSON 파일 로딩 후 df 만듬
# => KorQuAD_v1.0_dev.json 파일 로딩 하여, 각 항목별 리스트를 출력한후, contexts df, questions df를 만듬.
#-------------------------------------------------------------------------------------------------------
from myutils import read_korquad_v1_json, read_aihub_qua_json

# KorQuAD_v1.0 혹은 aihub 뉴스 기사 기계독해 데이터 QuA 파일을 불러옴.
jsonfile = './data/KorQuAD_v1.0_dev.json' # VL_unanswerable.json # VL_text_entailment.json # VL_span_inference.json # KorQuAD_v1.0_dev.json
contexts, questions, answers, contextids, qcontextids = read_korquad_v1_json(jsonfile) # read_aihub_qua_json(jsonfile)

# list들을 zip 으로 묶고, dataframe 생성함
# context, contextid를 묶어서 context df 만듬.
df_contexts = pd.DataFrame((zip(contexts, contextids)), columns = ['context','contextid'])

# question, answer, contextids를 묶어서 question df 만듬
df_questions = pd.DataFrame((zip(questions, answers, qcontextids)), columns = ['question','answer', 'contextid'])

In [ ]:
#-------------------------------------------------------------------------------------------------------
# 3. 문장(문서)들을 chunk(청크: 큰 덩어리)로 분리, 다시 분리된 chunks를 kss로 문장 분리해서 sentences 만듬, 이후 chunks와 sentences를 doc_sentences에 담음
# => 최대 512 단위로 서로 겹치게 청크 단위로 분리함.
#-------------------------------------------------------------------------------------------------------
from tqdm.notebook import tqdm
from myutils import sliding_window_tokenizer, split_sentences

#text = ["해외에서 데이터 무제한?? 뿐 아니라 음성과 문자 요금을 할인 받을 수 있는 ++요금제가 나왔다. 날씨는 좋다. lg유플러스는 중국, 일본, 홍콩, 싱가포르, 필리핀 등 아시아 8국을 대상으로~~ 무제한 데이터와 음성, 문자를 할인해주는 '스마트 로밍 요금제' 2종을 오는 28일부터 판매한다고 27일 밝혔다. . . 우선 '스마트 로밍음성'은 하루 기본료가 2000원으로 음성발신은 1분당 1000원이며, 문자메시지(sms)와 멀티미디어 문자메시지(mms)는 1건당 150원이다. 종전 500원에서 350원 인하했다. . . '스마트 로밍패키지'는 여기에 데이터 무제한 로밍서비스가 더해져 하루 기본료가 1만1000원이다. 기본료는 로밍 기간에 상관없이 사용한 당일에만 청구된다. . . 스마트 로밍 요금제는 일단 오는 7월 말까지 프로모션 형태로 제공하고 이후 정식 요금제로 추진한다는 계획이다. . . 아울러 해외에서 무제한 데이터 로밍 서비스와 데이터로밍 차단을 신청·해지할 수 있는 로밍 전용 모바일 홈페이지((이메일))도 운영한다. . . (이름) lg유플러스 글로벌로밍팀장은 '아시아 출(이름) 여행을 계획하고 있는 고객들이 저렴한 비용으로 안심하고 로밍 서비스를 사용할 수 있도록 이번 프로모션 요금제를 출시했다'며 '지속적으로 해외 로밍 이용 고객들의 편의성 증대를 위한상품을 준비할 예정'이라고 말했다. . . . . . . . # # #. . ■ 사진설명. . lg유플러스(부회장 (이름) / (이메일) )가 4월 28일부터 중국, 일본 등 아시아 8개국*을 대상으로 데이터와 음성 문자까지 할인해서 제공하는 스마트 로밍요금제(스마트 로밍음성/스마트 로밍패키지) 2종을 출시했다."]
#text =["오늘은 날씨가 ^^~~~~좋다. 내일은 비가 오고 춥겠다고 한다^^. 걱정이다. 오늘만큼만 매일 날씨가 좋으면 좋겠다.~~~~"]
contexts = df_contexts['context'].values.tolist()
start = time.time()


doc_sentences = []
tokenizer = word_embedding_model1.tokenizer
    
# 슬라이딩 윈도우 처리 후 chunks 리스트 만들고, 다시 chunks를 kss로 문장 분리해서, 최종 chunks와 sentences를 doc_sentences에 담음
if IS_SLIDING_WINDOW == True:
    for idx, context in enumerate(tqdm(contexts)):

        chunks = sliding_window_tokenizer(tokenizer = tokenizer, paragraph=context, window_size=WINDOW_SIZE, sliding_size=SLIDING_SIZE)
        sentences = split_sentences(paragraphs=chunks, sentences_split_num=1000, paragraphs_num=1000000, debug=False)

        # chunks 1차원 리스트[A,B,C]와 sentences 2차원 리스트[[a,b],[b,c],[d,e]] 를 합쳐서, doc_sentencs[A,a,b,B,b,c,C,d,e] 에 담음.
        arr = []
        [arr.extend([chunks[i], *sentences[i]]) for i in range(len(chunks))]

        # doc_sentences 리스트에 추가 
        doc_sentences.append(arr)
        
# 문장 분리해서 doc_sentences에 담음.
else:
    doc_sentences = split_sentences(paragraphs=contexts, sentences_split_num=1000, paragraphs_num=1000000, debug=False)

logger.info(f'*문장처리=>len:{len(doc_sentences[0])}, time:{time.time()-start:.4f}')
print(doc_sentences[0])

In [ ]:
from myutils import embed_text, onnx_embed_text

# 조건에 맞게 임베딩 처리하는 함수 
def embedding(paragrphs:list):
    if IS_ONNX_MODEL == True:
        embeddings = onnx_embed_text(model=onnx_model, tokenizer=onnx_tokenizer, paragraphs=paragrphs, token_embeddings=False)
    else:
        # 한 문단에 대한 40개 문장 배열들을 한꺼번에 임베딩 처리함
        embeddings = embed_text(model=bi_encoder1, paragraphs=paragrphs, return_tensor=False)  
    
    return embeddings
   

In [ ]:
# 분리된 문장들에 대해 클러스터링 실행
#-------------------------------------------------------------------------------------------------------
#문단에 문장들의 임베딩을 구하여 각각 클러스터링 처리함.
#-------------------------------------------------------------------------------------------------------
# Then, we perform k-means clustering using sklearn:
from sklearn.cluster import KMeans
from myutils import embed_text, fassi_index, clustering_embedding

#클러스터링 계수는 문단의 계수보다는 커야 함. 
assert num_clusters <= len(doc_sentences), f"num_clusters:{num_clusters} > len(doc_sentences):{len(doc_sentences)}"

start = time.time()

#-------------------------------------------------------------
# 각 문단의 문장들에 벡터를 구하고 리스트에 저장해 둠.
start = time.time()
cluster_list = []
faissindexlist = []

for i, sentences in enumerate(tqdm(doc_sentences)):
    embeddings = embedding(sentences)
    if i < 3:
        print(f'[{i}] sentences---------------------------')
        print(sentences)
        print(f'embeddings.shape: {embeddings.shape}')
   
    if IS_CLUSTERING == True:
        # 각 문단에 분할한 문장들의 임베딩 값을 입력해서 클러스터링 하고 평균값을 구함.
        emb = clustering_embedding(embeddings = embeddings, num_clusters= num_clusters, seed=seed)
    else:
        # 문장들에 대해 임베딩 값을 구하고 평균 구함.
        arr = np.array(embeddings).astype('float32')
        emb = arr.mean(axis=0).reshape(1,-1) #(128,) 배열을 (1,128) 형태로 만들기 위해 reshape 해줌
   
    #emb.astype('float16')
    if i == 0:
        print(f'emb:{emb[0]}')
        
    # Faiss index 생성하고 추가 
    index = fassi_index(embeddings=emb, method=faiss_index_method)
    faissindexlist.append(index)
  

print(f'*클러스터링 시간 : {time.time()-start:.4f}\n')
#-------------------------------------------------------------

In [ ]:
#-------------------------------------------------------------------------------------
# 쿼리문장 샘플링
#user_querys = ["독도에서 사고가 나서 실종자가 발생했다.", "오늘 날씨가 흐리고 비가 오겠다."]
#-------------------------------------------------------------------------------------
from myutils import df_sampling

# 쿼리 샘플링함.
if query_num == 0:   # query_num = 0 이면 모든 쿼리(5533개)
    user_querys = df_questions['question'].values.tolist()
else:   # query_num > 0이면 해당 계수만큼 랜덤하게 샘플링하여 쿼리 목록을 만듬.
    df_questions = df_sampling(df=df_questions, num=query_num, seed=seed)
    user_querys = df_questions['question'].values.tolist()
  
print(f'Query-----------------------------------------------------')
print(user_querys[0:3])

In [ ]:
#-------------------------------------------------------------------------------------
# 5. 쿼리 후 가장 유사한 문장(문서)의 sub 문장 평균 구함
# => 쿼리 문장에 임베딩값을 구하고, 이후 Faiss sub 문장 임베딩과 비교하여 가장 유사한 문장 avg_num개 를 찾고 평균을 구함
#    이후 평균값이 가장 큰 search_k 갯수 만 쿼리에 대한 예측 결과 리스트로 만듬
#-------------------------------------------------------------------------------------   
from myutils import df_sampling, sum_of_array_2d, index_of_list,split_sentence_list, clean_text

bi_predictions_list = []

embed_querys = embedding(user_querys)
        
if faiss_index_method == 0:
    faiss.normalize_L2(embed_querys)          # *cosine유사도 구할때는 반드시 normalize 처리함.

for embed_query in tqdm(embed_querys):
        embed_query = [embed_query]

        max_values_list = []
        for count, index in enumerate(faissindexlist):
            distance, idx = index.search(np.array(embed_query).astype("float32"), k=avg_num) # avg_num 계수 만큼 유사한 sub문장을 찾음
            avg_distance = distance.mean(axis=1) # 검색된 sub 문장들에 대해 평균을 구함
            max_values_list.append(avg_distance[0])

        # 문장별 스코어 최대값 리스트에서 최대값을 갖는 항목 search_k개 index만 출력 함.
        # faiss_index_method=0 => cosine유사도 구할때는 max 값을 갖는 index 출력함.(예: search_k=3일때, np.array([21,11,41,51,31]) 일때 출력 [3,2,4])
        # faiss_index_method=1 => 유클리드 거리 로 할때는 min 값을 갖는 index 출력함.(예: search_k=3일때, np.array([21,11,41,51,31]) 일때 출력 [1,0,4])
        indices = index_of_list(listdata=max_values_list, k=search_k, bmin=faiss_index_method)  

        # 예측검색결과 contextid값들을 리스트로 만듬.
        tmp_bi_predictions_list = []
        for indice in indices:
            tmp_bi_predictions_list.append(df_contexts["contextid"][indice])
            #print(f'*인덱스:{indice}/contextid:{df_contexts["contextid"][indice]}-----------------------------------------------------------')
            #print('*총합/평균 : {:.4f}/{:.4f}'.format(max_values_list[indice], float(max_values_list[indice]/len(mecab_query))))
            #print(df_contexts['context'][indice])
            #print(mecab_contexts[indice])

        # 2D 예측검색결과 리스트에 추가 
        bi_predictions_list.append(tmp_bi_predictions_list)

print(bi_predictions_list[0:3])


In [ ]:
#--------------------------------------------------------------------------------------------------
# 6. MRR 계산
# => 정답 리스트[2,3,1,4] 과 예측검색리스트[[1,2,5,1],[3,4,2,1],[6,5,4,1], [2,3,4,1]]를 입력하여 MRR 스코어 구함
##--------------------------------------------------------------------------------------------------
from myutils import mean_reciprocal_rank

# 정답, 여기서는 contextid를 리스트로 만듬.
ground_truths_list = df_questions['contextid'].values.tolist()
#print(f'gtlen:{len(ground_truths_list)}')
#print(ground_truths_list[0:9])

logger.info(f'--------------------------------------------------------------------------')
logger.info('json_file:{}'.format(jsonfile))
logger.info(f'faiss 인덱싱 방식: {faiss_index_method}(0=코사인 유사도, 1=유클리드 거리)/ONNX:{IS_ONNX_MODEL}')
logger.info('*avg_num:{}/search_k:{}/query_num:{}/out_dimension:{}'.format(avg_num, search_k, query_num, out_dimension))

# MRR 계산
bi_ranks, bi_score = mean_reciprocal_rank(ground_truths_list, bi_predictions_list)

# BI-MRR 출력
logger.info(f'----------------------------------------------------------------------------')
logger.info('*BI-ENCODER:{}'.format(bi_encoder_path))
logger.info('*BI-MRR:{:.4f}'.format(bi_score))
logger.info(f'*Ranks({len(bi_ranks)}):{bi_ranks[0:20]}')
    
# 검색 한 계슈
#logger.info(f'---------------------------------------------------------------------------')
search_count = 0
for item in bi_ranks:
    if item != 0:
        search_count += 1
    
logger.info('*검색률: {}/{}({:.2f}%)'.format(search_count, len(bi_ranks), (search_count/len(bi_ranks))*100))
logger.info(f'---------------------------------------------------------------------------')